Importing  Libraries


In [1]:
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD


In [2]:
lemmatizer = WordNetLemmatizer()


Loading Dataset



In [3]:
intents = json.loads(open('intents.json').read())


Preprocessing

In [4]:
words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignore_letters]
words = sorted(set(words))

classes = sorted(set(classes))


In [5]:
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(words, open('classes.pkl', 'wb'))


Spliting Data



In [6]:
training = []
output_empty = [0] * len(classes)

for document in documents:
    bag = []
    word_patterns = document[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

<ipython-input-6-eb9e65a60823>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


creating Model


In [7]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [8]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)


Epoch 1/200
10/10 [==============================] - 1s 3ms/step - loss: 2.3525 - accuracy: 0.0542
Epoch 2/200
10/10 [==============================] - 0s 3ms/step - loss: 2.2300 - accuracy: 0.2136
Epoch 3/200
10/10 [==============================] - 0s 3ms/step - loss: 2.1195 - accuracy: 0.3828
Epoch 4/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0927 - accuracy: 0.2713
Epoch 5/200
10/10 [==============================] - 0s 3ms/step - loss: 2.0981 - accuracy: 0.4084
Epoch 6/200
10/10 [==============================] - 0s 3ms/step - loss: 1.9723 - accuracy: 0.4963

Saving Model


In [9]:
model.save('Chatbot.h5')


In [10]:
def clean_up_sentences(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words


In [11]:
def bag_of_words(sentence):
    sentence_words = clean_up_sentences(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)



In [12]:
def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_TRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_TRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    return return_list


In [17]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            return result
    return "Sorry , I can't understand You"


In [19]:
while True:
    message = input("")
    if message == "quit":
        break
    print(f"User: {message}")
    ints = predict_class(message)
    res = get_response(ints, intents)
    print(f"Bot: {res}")

User: hi
Bot: Hey!
User: i need oxygen
Bot: Click Here : https://soumyajitnandi.000webhostapp.com/pages/oxygen.html  
 and find details about oxygen suppliers and you can also order from them
User: i need ambulance support
Bot: Click Here : https://soumyajitnandi.000webhostapp.com/pages/ambulance.html  
 and find details about oxygen suppliers and you can also order from them
